In [1]:
import dash
from dash import html, dcc, Input, Output, State, callback
import dash_bootstrap_components as dbc
from dash_iconify import DashIconify
import dash_mantine_components as dmc
import os

class FileTree:

    def __init__(self, filepath: os.PathLike):
        """
        Usage: component = FileTree('Path/to/my/File').render()
        """
        self.filepath = filepath

    def render(self) -> dmc.Accordion:
        return dmc.Accordion(
            self.build_tree(self.filepath, isRoot=True), multiple=True)

    def flatten(self, l):
        return [item for sublist in l for item in sublist]

    def make_file(self, file_name):
        return dmc.Text(
            [DashIconify(icon="akar-icons:file"), " ", file_name], style={"paddingTop": '5px'}
        )

    def make_folder(self, folder_name):
        return [DashIconify(icon="akar-icons:folder"), " ", folder_name]

    def build_tree(self, path, isRoot=False):
        d = []
        if os.path.isdir(path):
            children = self.flatten([self.build_tree(os.path.join(path, x))
                                    for x in os.listdir(path)])
            if isRoot:
                d.append(
                    dmc.AccordionItem([
                        dmc.AccordionControl(self.make_folder(os.path.basename(path))),
                        dmc.AccordionPanel(children=children)
                        ], value=str(path))
                )
            else:
                d.append(
                    dmc.Accordion(children=[
                        dmc.AccordionItem([
                            dmc.AccordionControl(self.make_folder(os.path.basename(path))),
                            dmc.AccordionPanel(children=children)
                            ], value=str(path))
                    ], multiple=True)
                )
        else:
            d.append(self.make_file(os.path.basename(path)))
        return d

In [2]:
import dagshub
from dagshub.common.api.repo import RepoAPI
from dagshub.data_engine.datasources import get_datasource
from io import BytesIO
from pathlib import Path
from PIL import Image
import requests

import plotly.express as px
import plotly.graph_objects as go

token = '9379eb5d9f33527968c7253f77074ac56d67d9e4'
headers = {"Authorization": f"Bearer {token}"}

In [3]:
repo_id, ds_id = 'Dean/COCO_1K', 'COCO_1K'
#repo_id, ds_id = 'OperationSavta/SavtaDepth', 'NYU_Depth_Processed'
#repo_id, ds_id = 'Omdena/TriesteItalyChapter_MappingSeagrassMeadows', 'Seagrass_ROI_Winter'
repo = RepoAPI(repo_id)

repo_info = repo.get_repo_info()

path_list = {}
for path in repo.list_path(''):
    path_list.setdefault(path.versioning, []).append(path)

ds = get_datasource(repo_id, ds_id)
query_all = ds.all()
df = query_all.dataframe
print(ds)

Accessing as zndr27

/home/zndr27/.cache/pypoetry/virtualenvs/repo-viz-Qeft-RkK-py3.12/lib/python3.12/site-packages/rich/live.py:229: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Datasource COCO_1K
	Repo: Dean/COCO_1K, path: repo://Dean/COCO_1K/main:data/images
	DatasourceQuery(as_of=None, time_zone=None, select=None, filter=Query: empty, order_by=None)
	Fields:
		media type (STRING)
		height (INTEGER)
		width (INTEGER)
		annotation (BLOB) with tags: {'annotation'}
		custom_metadata (STRING)
		size (INTEGER)
		categories (STRING)
		valid_datapoint (BOOLEAN)
		year (INTEGER)
		split (STRING)



In [30]:
import git
import os
from pathlib import Path
import requests
from sweetviz import analyze
import tempfile
from ydata_profiling import ProfileReport

# TODO start the dtale server...

data = {}

repo_owner, repo_name = repo_id.split("/")
dataset_id = ds_id

# Connect to the Dagshub Client
repo = RepoAPI(repo_id)
repo_info = repo.get_repo_info()

# Get the entire dataset
ds = get_datasource(repo_id, dataset_id)
query_all = ds.all()
df = query_all.dataframe

# File tree
tempdir = tempfile.TemporaryDirectory()
files_path = Path(tempdir.name) / repo_id
os.makedirs(files_path, exist_ok=True)
repo_git = git.Repo.clone_from(repo_info.clone_url, files_path)

# YData report
ydata_profile = ProfileReport(df, title="Profiling Report")
ydata_html = ydata_profile.to_html()

# Sweetviz report
sweetviz_report = analyze(df)
sweetviz_report.show_html(
    str(files_path / "sweetviz_report.html"), 
    open_browser=False, 
    layout="vertical",
)
with open(files_path / "sweetviz_report.html", "r") as f:
    sweetviz_html = f.read()

# Update app state on frontend
data |= {
    "repo_owner": repo_owner, 
    "repo_name": repo_name,
    "dataset_id": dataset_id,
    "dataframe": df.to_json(orient="columns"),
    "ydata_html": ydata_html,
    "sweetviz_html": sweetviz_html,
}

req = requests.post(
    "http://localhost:8080/create-df",
    json=data,
)

data |= req.json()

/home/zndr27/.cache/pypoetry/virtualenvs/repo-viz-Qeft-RkK-py3.12/lib/python3.12/site-packages/rich/live.py:229: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Summarize dataset:  74%|███████████████████▉       | 14/19 [00:00<00:00, 18.89it/s, Calculate auto correlation]/home/zndr27/.cache/pypoetry/virtualenvs/repo-viz-Qeft-RkK-py3.12/lib/python3.12/site-packages/ydata_profiling/model/correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'could not convert string to float: 'train'')
  warnings.warn(
Render HTML: 100%|███████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.95it/s]
Done! Use 'show' commands to display/save.   |█████████████████████████████████| [100%]   00:00 -> (00:00 left)

Report /tmp/tmpgde_mseq/Dean/COCO_1K/sweetviz_report.html was generated.


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [31]:
req

<Response [500]>

In [8]:
from dagshub.data_engine.datasources import get_datasources
ds_list = get_datasources(repo_id)

In [24]:
ds_list[0]._source

DatasourceState(repo='Dean/COCO_1K', name='COCO_1.1K_Demo', id=519, source_type=<DatasourceType.REPOSITORY: 'REPOSITORY'>, preprocessing_status=<PreprocessingStatus.READY: 'READY'>, path='repo://Dean/COCO_1K/main:data/images', client=<dagshub.data_engine.client.data_client.DataClient object at 0x7f40d023d730>, repoApi=<dagshub.common.api.repo.RepoAPI object at 0x7f40d023deb0>, metadata_fields=[annotation (BLOB) with tags: {'annotation'}, year (INTEGER), split (STRING), categories (STRING), valid_datapoint (BOOLEAN), size (INTEGER)], _revision='main')

In [4]:
repo_info

RepoAPIResponse(id=20451, owner=UserAPIResponse(id=5, login='Dean', full_name='Dean Pleban', avatar_url='https://dagshub.com/avatars/5', public_email='', website='https://dagshub.com/Dean', company='DagsHub', description='', username='Dean'), name='COCO_1K', full_name='Dean/COCO_1K', description='1000 Images from COCO dataset with polygon segmentation', private=False, fork=False, parent=None, empty=False, mirror=False, size=1490944, html_url='https://dagshub.com/Dean/COCO_1K', ssh_url='ssh://git@dagshub.com:2200/Dean/COCO_1K.git', clone_url='https://dagshub.com/Dean/COCO_1K.git', website='', stars_count=7, forks_count=27, watchers_count=2, open_issues_count=0, default_branch='main', created_at='2023-07-29T14:28:49Z', updated_at='2024-05-29T13:57:41Z', permissions={'admin': False, 'push': False, 'pull': True})

In [5]:
search_value = 'COCO'
response = requests.get(
    f"https://dagshub.com/api/v1/repos/search?q={search_value}&uid=0&limit=50&page=1",
    headers=headers,
)

In [8]:
dir(response)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

In [6]:
[_.get('full_name') for _ in response.json().get('data')]

['Simon/COCO-2017',
 'radoudounousta/coco',
 'dudifrid/COCO',
 'xiang-wuu/coco128',
 'Devolta05/Occluded_COCOC_Dataset',
 'DagsHub-Datasets/fast-ai-coco-dataset',
 'vysakhramkrishnan7/COCO_Dataset_Semantic_Segmentation',
 'noa/annotate-coco',
 'Nikitha-Narendra/Transfer-Annotation-COCO',
 'ch.amnuay/COCO-2017',
 'Dean/COCO8',
 'Dean/COCO8_Demo',
 'Nikitha-Narendra/COCO8',
 'Dean/COCO_1K',
 'BrentMat/COCO_1K',
 'ij451r/COCO_1K',
 'yonomitt/COCO_1K',
 'eugenia.anello/COCO_1K',
 'erow/COCO_1K',
 'wyzhu234/COCO_1K',
 'imentivai/COCO_1K',
 'pragadeeshraju/COCO_1K',
 'benjipeng/COCO_1K_learnfrom_demo',
 'nikhilkshaji/COCO_1K',
 'nirbarazida/COCO8',
 'nirbarazida/COCO_1K',
 'Dean/DagsHubDrive',
 'idonov/COCO_1K',
 'tiendd.public/COCO_1K',
 'cyberbeam524/COCO_1K',
 'GenceErdagi/COCO_1K',
 'Simon/COCO_1K',
 'anna/COCO_1K_fork_test',
 'prakugenius/COCO_1K',
 'diabyanas/cocoa-disease-detection',
 'diabyanas/cocoa-diseases-detection',
 'jaimin-simform/autolabel1',
 'avi/COCO_1K',
 'jcodeing/COCO_1

In [16]:
"""
A simple app demonstrating how to manually construct a navbar with a customised
layout using the Navbar component and the supporting Nav, NavItem, NavLink,
NavbarBrand, and NavbarToggler components.

Requires dash-bootstrap-components 0.3.0 or later
"""

import dash
import dash_bootstrap_components as dbc
from dash import Input, Output, State, html

PLOTLY_LOGO = "https://images.plot.ly/logo/new-branding/plotly-logomark.png"

app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
# try running the app with one of the Bootswatch themes e.g.
# app = dash.Dash(external_stylesheets=[dbc.themes.JOURNAL])
# app = dash.Dash(external_stylesheets=[dbc.themes.SKETCHY])

# make a reuseable navitem for the different examples
nav_item = dbc.NavItem(dbc.NavLink("Link", href="#"))

# make a reuseable dropdown for the different examples
dropdown = dbc.DropdownMenu(
    children=[
        dbc.DropdownMenuItem("Entry 1"),
        dbc.DropdownMenuItem("Entry 2"),
        dbc.DropdownMenuItem(divider=True),
        dbc.DropdownMenuItem("Entry 3"),
    ],
    nav=True,
    in_navbar=True,
    label="Menu",
)

# this is the default navbar style created by the NavbarSimple component
default = dbc.NavbarSimple(
    children=[nav_item, dropdown],
    brand="Default",
    brand_href="#",
    sticky="top",
    className="mb-5",
)

# here's how you can recreate the same thing using Navbar
# (see also required callback at the end of the file)
custom_default = dbc.Navbar(
    dbc.Container(
        [
            dbc.NavbarBrand("Custom default", href="#"),
            dbc.NavbarToggler(id="navbar-toggler1"),
            dbc.Collapse(
                dbc.Nav(
                    [nav_item, dropdown], className="ms-auto", navbar=True
                ),
                id="navbar-collapse1",
                navbar=True,
            ),
        ]
    ),
    className="mb-5",
)


# this example that adds a logo to the navbar brand
logo = dbc.Navbar(
    dbc.Container(
        [
            html.A(
                # Use row and col to control vertical alignment of logo / brand
                dbc.Row(
                    [
                        dbc.Col(html.Img(src=PLOTLY_LOGO, height="30px")),
                        dbc.Col(dbc.NavbarBrand("Logo", className="ms-2")),
                    ],
                    align="center",
                    className="g-0",
                ),
                href="https://plotly.com",
                style={"textDecoration": "none"},
            ),
            dbc.NavbarToggler(id="navbar-toggler2", n_clicks=0),
            dbc.Collapse(
                dbc.Nav(
                    [nav_item, dropdown],
                    className="ms-auto",
                    navbar=True,
                ),
                id="navbar-collapse2",
                navbar=True,
            ),
        ],
    ),
    color="dark",
    dark=True,
    className="mb-5",
)

# this example has a search bar and button instead of navitems / dropdowns
search_navbar = dbc.Navbar(
    dbc.Container(
        [
            dbc.NavbarBrand("Search", href="#"),
            dbc.NavbarToggler(id="navbar-toggler3"),
            dbc.Collapse(
                dbc.Row(
                    [
                        dbc.Col(
                            dbc.Input(type="search", placeholder="Search")
                        ),
                        dbc.Col(
                            dbc.Button(
                                "Search", color="primary", className="ms-2"
                            ),
                            # set width of button column to auto to allow
                            # search box to take up remaining space.
                            width="auto",
                        ),
                    ],
                    # add a top margin to make things look nice when the navbar
                    # isn't expanded (mt-3) remove the margin on medium or
                    # larger screens (mt-md-0) when the navbar is expanded.
                    # keep button and search box on same row (flex-nowrap).
                    # align everything on the right with left margin (ms-auto).
                    className="g-0 ms-auto flex-nowrap mt-3 mt-md-0",
                    align="center",
                ),
                id="navbar-collapse3",
                navbar=True,
            ),
        ]
    ),
    className="mb-5",
)

# custom navbar based on https://getbootstrap.com/docs/4.1/examples/dashboard/
dashboard = dbc.Navbar(
    dbc.Container(
        [
            dbc.Col(dbc.NavbarBrand("Dashboard", href="#"), sm=3, md=2),
            dbc.Col(dbc.Input(type="search", placeholder="Search here")),
            dbc.Col(
                dbc.Nav(
                    dbc.Container(dbc.NavItem(dbc.NavLink("Sign out"))),
                    navbar=True,
                ),
                width="auto",
            ),
        ],
    ),
    color="dark",
    dark=True,
)

app.layout = html.Div(
    [default, custom_default, logo, search_navbar, dashboard]
)


# we use a callback to toggle the collapse on small screens
def toggle_navbar_collapse(n, is_open):
    if n:
        return not is_open
    return is_open


# the same function (toggle_navbar_collapse) is used in all three callbacks
for i in [1, 2, 3]:
    app.callback(
        Output(f"navbar-collapse{i}", "is_open"),
        [Input(f"navbar-toggler{i}", "n_clicks")],
        [State(f"navbar-collapse{i}", "is_open")],
    )(toggle_navbar_collapse)

if __name__ == "__main__":
    app.run_server(debug=True, port=8052)
    p

In [5]:
import git
import contextlib
import tempfile

#@contextlib.contextmanager
#def temp_repo(url: AnyPath) -> Generator[git.Repo, None, None]:
#    """
#    Returns a git repo cloned to a temp directory
#    """
#    with tempfile.TemporaryDirectory() as tmpdirname:
#        yield git.Repo.clone_from(url, tmpdirname)
td = tempfile.TemporaryDirectory()

In [6]:
repo_git = git.Repo.clone_from(repo_info.clone_url, td.name)

In [7]:
os.listdir(td.name)

['.dvcignore',
 'custom_coco.yaml',
 'data.dvc',
 '.gitignore',
 'README.md',
 'de_walkthrough.ipynb',
 '.git',
 'ls_backend',
 'utils',
 'Makefile',
 'resources',
 'requirements.txt',
 '.dvc']

In [9]:
from pathlib import Path
from natsort import natsorted
path = Path(td.name)
folders = natsorted([x for x in path.iterdir() if x.is_dir()])
files = natsorted([x for x in path.iterdir() if not x.is_dir()])
result = folders + files

In [10]:
result

[PosixPath('/tmp/tmppeac6yry/.dvc'),
 PosixPath('/tmp/tmppeac6yry/.git'),
 PosixPath('/tmp/tmppeac6yry/ls_backend'),
 PosixPath('/tmp/tmppeac6yry/resources'),
 PosixPath('/tmp/tmppeac6yry/utils'),
 PosixPath('/tmp/tmppeac6yry/.dvcignore'),
 PosixPath('/tmp/tmppeac6yry/.gitignore'),
 PosixPath('/tmp/tmppeac6yry/Makefile'),
 PosixPath('/tmp/tmppeac6yry/README.md'),
 PosixPath('/tmp/tmppeac6yry/custom_coco.yaml'),
 PosixPath('/tmp/tmppeac6yry/data.dvc'),
 PosixPath('/tmp/tmppeac6yry/de_walkthrough.ipynb'),
 PosixPath('/tmp/tmppeac6yry/requirements.txt')]

In [11]:
df

,path,datapoint_id,dagshub_download_url,annotation,categories,height,media type,size,split,valid_datapoint,width,year
0,train/000000005105.jpg,44457250,https://dagshub.com/api/v1/repos/Dean/COCO_1K/...,a91ae4c7313ded6f3d4d1c8690d6270b0a1e50c0f6414b...,"bench, dog, sports ball, person, car, surfboard",480,image/jpeg,127731,train,True,640,2017
1,train/000000000722.jpg,44457251,https://dagshub.com/api/v1/repos/Dean/COCO_1K/...,27af1c02c73bcf9a7fa6ca71e3b8593f0f60cc25fea9b0...,"car, backpack, kite, person",457,image/jpeg,130509,train,True,640,2017
2,train/000000001404.jpg,44457252,https://dagshub.com/api/v1/repos/Dean/COCO_1K/...,dbaaf7176c7ea5e37427a878cd05006cffd8b1071c9ead...,"teddy bear, couch, person",640,image/jpeg,149037,train,True,478,2017
3,train/000000001681.jpg,44457253,https://dagshub.com/api/v1/repos/Dean/COCO_1K/...,b1221a645f24134449cecf7c8b2eed9113c612cdf2e876...,"book, clock, plate",333,image/jpeg,135878,train,True,500,2017
4,train/000000003084.jpg,44457254,https://dagshub.com/api/v1/repos/Dean/COCO_1K/...,c6cba8de8ef2d3d36bb3965c30aac1b10b7a41dd08ae10...,clock,480,image/jpeg,152442,train,True,640,2017
...,...,...,...,...,...,...,...,...,...,...,...,...
1194,train/new_data/000000001076.jpg,46503740,https://dagshub.com/api/v1/repos/Dean/COCO_1K/...,bf4d0d261355ab18a749938bdee742f8e3dd9256982acb...,"frisbee, person",428,image/jpeg,153605,train,True,640,2017
1195,train/new_data/000000000229.jpg,46503744,https://dagshub.com/api/v1/repos/Dean/COCO_1K/...,f6d648b2e04b18a61a0ab1b819f81c92a42ec4be4711be...,giraffe,428,image/jpeg,106330,train,True,640,2017
1196,train/new_data/000000000492.jpg,46503745,https://dagshub.com/api/v1/repos/Dean/COCO_1K/...,a461c832898cf5e347ca932712b867932911f5ed490160...,"person, oven",383,image/jpeg,261056,train,True,640,2017
1197,train/new_data/000000001251.jpg,46503746,https://dagshub.com/api/v1/repos/Dean/COCO_1K/...,8af4521de762b4af822526604b66107fad74a1f3141d0a...,"person, sink, couch, bottle",428,image/jpeg,163447,train,True,640,2017


In [8]:
file_tree = FileTree(td.name).render()

In [9]:
file_tree

Accordion(children=[AccordionItem(children=[AccordionControl([DashIconify(icon='akar-icons:folder'), ' ', 'tmpdwyilzh1']), AccordionPanel([Text(children=[DashIconify(icon='akar-icons:file'), ' ', '.dvcignore'], style={'paddingTop': '5px'}), Text(children=[DashIconify(icon='akar-icons:file'), ' ', 'custom_coco.yaml'], style={'paddingTop': '5px'}), Text(children=[DashIconify(icon='akar-icons:file'), ' ', 'data.dvc'], style={'paddingTop': '5px'}), Text(children=[DashIconify(icon='akar-icons:file'), ' ', '.gitignore'], style={'paddingTop': '5px'}), Text(children=[DashIconify(icon='akar-icons:file'), ' ', 'README.md'], style={'paddingTop': '5px'}), Text(children=[DashIconify(icon='akar-icons:file'), ' ', 'de_walkthrough.ipynb'], style={'paddingTop': '5px'}), Accordion(children=[AccordionItem(children=[AccordionControl([DashIconify(icon='akar-icons:folder'), ' ', '.git']), AccordionPanel([Text(children=[DashIconify(icon='akar-icons:file'), ' ', 'index'], style={'paddingTop': '5px'}), Text(ch

In [10]:
accord_layout = dmc.Accordion(
    disableChevronRotation=True,
    children=[
        dmc.AccordionItem(
            [
                dmc.AccordionControl(
                    "Personal Information",
                    icon=DashIconify(
                        icon="tabler:user",
                        color=dmc.DEFAULT_THEME["colors"]["blue"][6],
                        width=20,
                    ),
                ),
                dmc.AccordionPanel("some content"),
            ],
            value="info",
        ),
        dmc.AccordionItem(
            [
                dmc.AccordionControl(
                    "Shipping Address",
                    icon=DashIconify(
                        icon="tabler:map-pin",
                        color=dmc.DEFAULT_THEME["colors"]["red"][6],
                        width=20,
                    ),
                ),
                dmc.AccordionPanel("some content"),
            ],
            value="addr",
        ),
        dmc.AccordionItem(
            [
                dmc.AccordionControl(
                    "Confirmation",
                    icon=DashIconify(
                        icon="tabler:circle-check",
                        color=dmc.DEFAULT_THEME["colors"]["green"][6],
                        width=20,
                    ),
                ),
                dmc.AccordionPanel("some content"),
            ],
            value="focus",
        ),
    ],
)

In [3]:
import dash_mantine_components as dmc
accord_layout = dmc.Accordion(
    children=[
        dmc.AccordionItem(
            [
                dmc.AccordionControl("Personal Information"),
                dmc.AccordionPanel("some content"),
            ],
            value="info",
        ),
    ],
)

In [4]:
from dash import Dash, html
app = Dash(__name__)
app.layout = dmc.MantineProvider(
    [
        'wtf',
        accord_layout,
        'bruh',
    ],
)
app.run()

In [1]:
import dash
dir(dash._dash_renderer)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__version__',
 '_available_react_versions',
 '_available_reactdom_versions',
 '_env_react_version',
 '_js_dist',
 '_js_dist_dependencies',
 '_set_react_version',
 'os']

In [27]:
#dash._dash_renderer._set_react_version('18.2.0')

In [3]:
#annotations = query_all.get_blob_fields(
#    'annotation', 
#    load_into_memory=True,
#    cache_on_disk=False,
#    num_proc=32,
#)
#
#import itertools
#from collections import Counter
#counts = Counter(itertools.chain(*(x.split(', ') for x in df['categories'])))
#
#px.bar(x=list(counts.keys()), y=list(counts.values()))

In [3]:
import pandas as pd
from ydata_profiling import ProfileReport
profile = ProfileReport(df, title="Profiling Report")

In [21]:
import sweetviz as sv
my_report = sv.analyze(df)

                                             |                                        | [  0%]   00:00 -> (? l…

In [23]:
dir(my_report)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_association_graphs',
 '_association_graphs_compare',
 '_associations',
 '_associations_compare',
 '_features',
 '_jupyter_html',
 '_page_html',
 '_target',
 'associations_html_compare',
 'associations_html_source',
 'compare_name',
 'corr_warning',
 'dataframe_summary_html',
 'generate_comet_friendly_html',
 'get_predetermined_type',
 'get_sanitized_bool_series',
 'get_target_type',
 'get_type',
 'get_what_influences_me',
 'graph_legend',
 'log_comet',
 'num_summaries',
 'process_associations',
 'progress_bar',
 'run_id',
 'sanitize_bool',
 'show_html',
 'show_notebook',


In [26]:
my_report.show_html()

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


Couldn't find a suitable web browser!

Set the BROWSER environment variable to your desired browser.



Opening "/home/zndr27/repos/yeet/SWEETVIZ_REPORT.html" with LibreOffice Writer  (text/html)


In [3]:
import dtale
d = dtale.show(df, host='localhost', subprocess=True)

In [4]:
d

In [4]:
dir(d)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_iframe',
 '_data_id',
 '_main_url',
 '_notebook_handle',
 '_url',
 'adjust_cell_dimensions',
 'app_root',
 'build_main_url',
 'cleanup',
 'data',
 'get_corr_matrix',
 'get_pps_matrix',
 'get_settings',
 'is_proxy',
 'is_up',
 'kill',
 'main_url',
 'notebook',
 'notebook_charts',
 'notebook_correlations',
 'offline_chart',
 'open_browser',
 'started_with_open_browser',
 'update_id',
 'update_settings',
 'view_data']

In [5]:
d.__dict__

{'_data_id': '1',
 '_url': 'http://localhost:40000',
 '_notebook_handle': None,
 'started_with_open_browser': False,
 'is_proxy': False,
 'app_root': None}